In [ ]:
metaquast mcav.contigs.fa -o quast_output
#metaquast - quality assessment of metagenomic reads, no reference genome included

https://quast.sourceforge.net/docs/manual.html#sec1
..how to interpret quality results?
- check how many large contigs you have (>1000 bp)
- did not map to reference genome.
- right now just helpful to see length and quality of contigs, maybe can reassess after mapping back to metagenome?
Cite metaquast: https://quast.sourceforge.net/publications.html

In [ ]:
conda create -n anvio-7.1
#dir=/home/brooke_sienkiewicz_student_uml_edu/.conda/envs/anvio-7.1


Mapping
- reassembling contigs back to metagenome 
- only use contigs >1000 bp

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=50G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 20:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o slurm-%j.out  # %j = job ID

module load miniconda/22.11.1-1
conda activate anvio-7.1

SAMPLENAME=mcav
READSPATH=/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/trimmed
CONTIGPATH=/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/results
CONTIGFILE=mcav.contigs.fa

anvi-script-reformat-fasta $CONTIGPATH/$CONTIGFILE -o $CONTIGPATH/$SAMPLENAME.contigs-fixed.fa -l 1000 --simplify-names
#fixes deflines for later and filters on size (set to 1000 bp)

FIXEDCON="$SAMPLENAME".contigs-fixed.fa

bowtie2-build $CONTIGPATH/$FIXEDCON ../working/"$SAMPLENAME"_contigs
#this builds an index of your contigs, which only needs to happen once

for f in <T1_12_2022> <T1_13_2022> <T1_16_2019> <T1_20_2019> <T1_24_2019> <T1_40_2022> <T1_57_2022> <T1_70_2022> <T2_10_2022> <T2_16_2019> <T3_13_2022> <T3_14_2019> <T3_15_2019> <T3_19_2022> <T3_1_2019> <T3_40_2022> <T3_48_2022> <T3_49_2022> <T3_51_2022> <T3_60_2022> <T3_8_2019> <T3_9_2019>
do
bowtie2 --threads 11 -x ../working/"$SAMPLENAME"_contigs -1 $READSPATH/"$f”_MCAV_R1_001_val_1.fq -2 $READSPATH/"$f”_MCAV_R2_001_val_2.fastq -S ../working/"$f".sam
#this creates an alignment of your reads to your contigs and collects that in a .sam file

samtools view -F 4 -b -S ../working/"$f".sam -o ../working/"$f"-RAW.bam
#this converts your sam file to a bam file, but its neither sorted nor indexed, so we use an Anvi'O script to do so:

anvi-init-bam ../working/"$f"_-RAW.bam -o ../results/index/"$f".bam
#as said, this is how you index and sort your bam file

rm ../data/working/"$f"-RAW.bam
done



#bash script: mapping.txt